In [6]:
from algorithms_pca.matrix import Matrix

In [7]:
A = Matrix([[4, 7], [2, 6]])
B = Matrix([[3, 8], [5, 1]])
print("A + B =", A + B)
print("A - B =", A - B)
print("A * B (elem) =", A * B)
print("A.matmul(B) =", A.matmul(B))
print("A.T =", A.T)
print("det(A) =", A.determinant())
print("inv(A) =", A.inverse())
print("rank(A) =", A.rank())
print("trace(A) =", A.trace())
P, L, U = A.lu()
print("P, L, U =", P, L, U)
x = A.solve(Matrix([[1], [0]]))
print("Solution x to Ax=b =", x)

A + B = Matrix([[7, 15], [7, 7]])
A - B = Matrix([[1, -1], [-3, 5]])
A * B (elem) = Matrix([[12, 56], [10, 6]])
A.matmul(B) = Matrix([[47, 39], [36, 22]])
A.T = Matrix([[4, 2], [7, 6]])
det(A) = 10.0
inv(A) = Matrix([[0.6000000000000001, -0.7000000000000001], [-0.2, 0.4]])
rank(A) = 2
trace(A) = 10
P, L, U = Matrix([[1.0, 0.0], [0.0, 1.0]]) Matrix([[1.0, 0.0], [0.0, 1.0]]) Matrix([[4, 7], [0.0, 2.5]])
Solution x to Ax=b = Matrix([[0.25], [0.0]])
